In [1]:
import tweepy
import json
import pandas as pd
from collections import Counter
from pprint import pprint 
import operator 
import time
from itertools import islice

In [2]:
#PUNTO A

In [3]:
auth = tweepy.OAuthHandler("MwBn5s2sjQ3vrRYMXuc9BtYfw", "36Cc2pA4NN89wTXs3LgUCxSznlpa4v8PoPYw6WTLuH1BF8V5vC")
auth.set_access_token("1037434933835378694-bUg0vb50KLHiaC78YtIa2pxSdnEytU", "VHvbqFILbnDgv2xZesppbs7f1Fk2QPDBIJFB8KoYkUCUH")
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [4]:
me = api.me() #Para futuro uso

In [5]:
trends_mundiales = api.trends_place(1) 


In [6]:
nombre_trends = set([trend['name'] for trend in trends_mundiales[0]['trends']])

In [7]:
top_trends = [] #para uso en el punto siguiente

In [8]:
for i, val in enumerate(islice(nombre_trends, 3)):
    top_trends.append(val)
    print(val)
    

#BatmanDay
#InterParma
#BANvSL


In [9]:
#FIN PUNTO A

In [10]:
#PUNTO B

In [11]:
for t in top_trends:    
    print("########### " + t + " ###########" )
    search_result = api.search(q=t)    
    for i in search_result['statuses']:
        (i['user']['screen_name'])   #para printear los usuarios 

########### #BatmanDay ###########
########### #InterParma ###########
########### #BANvSL ###########


In [12]:
#FIN PUNTO B

In [13]:
#PUNTO C

In [14]:
for t in top_trends:  
    print("########### " + t + " ###########" )
    cnt = Counter() 
    tweets = api.search(q=t)    
    for i in tweets['statuses']:                
        for word in i["text"].split(): 
            cnt[word.lower()] += 1
    sorted_cnt = sorted(cnt.items(),key=operator.itemgetter(1),reverse=True) #python2: iteritems
    pprint(sorted_cnt[:5]) # top 5 palabras 
    

########### #BatmanDay ###########
[('#batmanday', 12), ('rt', 11), ('to', 8), ('the', 6), ('happy', 5)]
########### #InterParma ###########
[('#interparma', 13), ('rt', 10), ('di', 9), ('il', 6), ('che', 5)]
########### #BANvSL ###########
[('#banvsl', 11), ('after', 7), ('rt', 7), ('sri', 6), ('lanka', 6)]


In [15]:
#FIN PUNTO C

In [16]:
#PUNTO 2_1_PASIVO

In [17]:
#Usuario -> seguidores -> seguidores a su vez, >10
#herramienta gráficos: d3js
#seguidores (100, 150)
# para agregar python 3 kernel : python3 -m ipykernel install --user

In [18]:
def seguidores(usuario):    
    #ids = []
    f = []
    #cant = 1
    for page in tweepy.Cursor(api.followers_ids, screen_name=usuario).pages():                 
        #f.append(page['users'][0]['screen_name'])
        f.append(page)
        #ids.extend(page)
        time.sleep(60)        
        '''
        if cant >10:
            break
        else:
            cant+=1        
        '''
    return f
            

In [19]:
red_usuarios = {} 
search_result = api.search(q=top_trends[1], count=200)    
for i in search_result['statuses']:
    if(i['user']['followers_count'])>10:              
        usuario_X = i['user']['screen_name']
        print(usuario_X)
        break
f= seguidores(usuario_X)


JaredPollard28


In [22]:
cant = 1
for v in f:    
    for vids in v['ids']:
        u = api.get_user(vids) #se obtiene el usuario.
        if u['followers_count']>10:
            print(u['screen_name'])
            cant+=1
            if cant >10:
                break
        
        

BULLETlN
tipsterreviews
juvephotos_
Mz_Rej
merchanttunes
SocialAdsPro
RiRiStarr
reggaeyardvybz1
KobbyHunt
xoxo_fii
